In [57]:
import os
from torch.utils.data import DataLoader,Dataset, random_split, Subset
from matplotlib import pyplot as plt
import pandas as pd
import fnmatch
import torch
import spacy

In [58]:
def _find_files(directory, pattern='*.final'):
        """Recursively find all files matching the pattern."""
        file_path_list = []
        for root, dirnames, filenames in os.walk(directory):
            for filename in fnmatch.filter(filenames, pattern):
                file_path_list.append(os.path.join(root, filename))
        return file_path_list

In [59]:
file_list = _find_files('../Dataset/')

In [12]:
# file = '../Dataset/dev/2019/2019-45-2-293-337.final'
# token_label_list = []
# with open(file,'r',errors='replace') as f:
#         for token_label in f.read().splitlines():
#             token_label_list.append(token_label)

# Building Vocabulary

In [67]:
word_list = ['UNK','PAD']
total_word = []
token_label_list = []
for file in file_list:
#     print(file)
    #becasue some file have issue with encoding. so we just replace it to "?" if they can encode it.
    with open(file,'r', errors='replace') as f:
        for token_label in f.read().splitlines():
#             print(token_label)
            # select exclude empty line because some annotator not follow this rule.
            if token_label != '':
                # some line does not use tab '\t'
                if not token_label.__contains__('\t'):
    #                 print(token_label)
                    token_label_list.append(token_label)
                    #split word and term by space
                    token = token_label.split()[0]
                    word_list.append(token.lower())
                elif token_label.__contains__('\t'):
                    token_label_list.append(token_label)
                    #split word and term by tab
                    token = token_label.split('\t')[0]
                    word_list.append(token.lower())
                else:
                    print('# '+str(token_label)+file+' #')


In [68]:
len(word_list)

26748

In [69]:
# len(word_list) services
word_list

['UNK',
 'PAD',
 'translation',
 'models',
 'used',
 'for',
 'statistical',
 'machine',
 'translation',
 'are',
 'compiled',
 'from',
 'parallel',
 'corpora',
 'that',
 'are',
 'manually',
 'translated',
 '.',
 'the',
 'common',
 'assumption',
 'is',
 'that',
 'parallel',
 'texts',
 'are',
 'symmetrical',
 ':',
 'the',
 'direction',
 'of',
 'translation',
 'is',
 'deemed',
 'irrelevant',
 'and',
 'is',
 'consequently',
 'ignored',
 '.',
 'much',
 'research',
 'in',
 'translation',
 'studies',
 'indicates',
 'that',
 'the',
 'direction',
 'of',
 'translation',
 'matters',
 ',',
 'however',
 ',',
 'as',
 'translated',
 'language',
 '(',
 'translationese',
 ')',
 'has',
 'many',
 'unique',
 'properties',
 '.',
 'it',
 'has',
 'already',
 'been',
 'shown',
 'that',
 'phrase',
 'tables',
 'constructed',
 'from',
 'parallel',
 'corpora',
 'translated',
 'in',
 'the',
 'same',
 'direction',
 'as',
 'the',
 'translation',
 'task',
 'outperform',
 'those',
 'constructed',
 'from',
 'corpora',
 

In [70]:
vocab = {}

for index,word in enumerate(word_list):
    vocab[word] = index

In [71]:
len(vocab)

3367

In [75]:
vocab

{'UNK': 0,
 'PAD': 1,
 'translation': 26240,
 'models': 26459,
 'used': 26665,
 'for': 26731,
 'statistical': 26596,
 'machine': 26239,
 'are': 26699,
 'compiled': 23547,
 'from': 26291,
 'parallel': 23806,
 'corpora': 26628,
 'that': 26740,
 'manually': 24604,
 'translated': 23642,
 '.': 26747,
 'the': 26732,
 'common': 22420,
 'assumption': 7526,
 'is': 26677,
 'texts': 24585,
 'symmetrical': 27,
 ':': 26510,
 'direction': 230,
 'of': 26734,
 'deemed': 2938,
 'irrelevant': 35,
 'and': 26745,
 'consequently': 22495,
 'ignored': 20444,
 'much': 26212,
 'research': 26366,
 'in': 26737,
 'studies': 26170,
 'indicates': 13207,
 'matters': 52,
 ',': 26655,
 'however': 22521,
 'as': 26535,
 'language': 26350,
 '(': 26412,
 'translationese': 214,
 ')': 26414,
 'has': 25812,
 'many': 25638,
 'unique': 22234,
 'properties': 22511,
 'it': 26630,
 'already': 69,
 'been': 25813,
 'shown': 24472,
 'phrase': 22854,
 'tables': 245,
 'constructed': 25100,
 'same': 24042,
 'task': 25874,
 'outperform'

# Preparing dataset

In [87]:
# check length of all abstract
max_seq = 0
seq = []
for abstract in file_list:
#     print(abstract)
    with open(abstract,'r', errors='replace') as f:
        for token_label in f.read().splitlines():
            # select exclude empty line because some annotator not follow this rule.
            if token_label != '':
                # some line does not use tab '\t'
                if not token_label.__contains__('\t'):
    #                 print(token_label)
                    #split word and term by space
                    token = token_label.split()[0]
                elif token_label.__contains__('\t'):
                    #split word and term by tab
                    token = token_label.split('\t')[0]
                else:
                    print('# '+str(token_label)+abstract+' #')
                
                if token == '.':
                    seq = []
                else:
                    seq.append(token)
                    
                if len(seq)> max_seq:
                    max_seq = len(seq)
#                     print(max_seq)
print(max_seq)
# becasue the highest seq length is around 175 token. so we decide we chose the sequence length is only 10 tokens.

175


In [88]:
file_train = _find_files('../Dataset/train')

In [91]:
train_pair = []

# create sequence of token with sequnce leght of 10 tokens. and sequence of label with 10 labels
for train_file in file_train:
    with open(train_file) as f:
        for token_label in f.read().splitlines():
            if token_label != '':
                if not token_label.__contains__('\t'):
                    token = token_label.split()[0]
                    label = token_label.split()[1]
#                     train_pair.append([token,label]) 
                elif token_label.__contains__('\t'):
                    token = token_label.split('\t')[0]
                    label = token_label.split('\t')[1]
#                     train_pair.append([token,label])
                else:
                     print('# '+str(token_label)+train_file+' #')
            
            train_pair.append([token.lower(),label])

# with open(train_sentences_file) as f:
#     for sentence in f.read().splitlines():
#         #replace each token by its index if it is in vocab
#         #else use index of UNK
#         s = [vocab[token] if token in self.vocab 
#             else vocab['UNK']
#             for token in sentence.split(' ')]
#         train_sentences.append(s)

# with open(train_labels_file) as f:
#     for sentence in f.read().splitlines():
#         #replace each label by its index
#         l = [tag_map[label] for label in sentence.split(' ')]
#         train_labels.append(l)  

In [93]:
len(train_pair)

21979

In [94]:
#Prepare at batch of sequence
seq_token_list = []
seq_token = []
seq_label_list = []
seq_label = []
for token_label in train_pair:
    if (len(seq_token) == 10) and (len(seq_label) == 10):
        seq_token_list.append(seq_token)
        seq_label_list.append(seq_label)
        seq_token = []
        seq_label = []
    else:
        seq_token.append(token_label[0])
        seq_label.append(token_label[1])
        

In [97]:
len(seq_token_list[-2])

10

# New code

In [98]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

max_len = 16
batch_size = 64
embed_size = 300
hidden_size = 128

In [ ]:
# On crée un tableau de dimension len(text) × max_len, et on le remplit de 0
X = torch.zeros(len(texts), max_len, dtype=torch.long)

# On remplit le tensor ligne par ligne avec nos textes convertis en nombres entiers
for i, int_text in enumerate(int_tokens): # Pour chaque texte
    if len(int_text) < max_len: # Si le texte est trop court
        int_text = int_text + [len(token2int)] * (max_len - len(int_text)) # Alors on lui rajoute des tokens vides jusqu'à
        # atteindre la bonne longueur
    X[i] = torch.LongTensor(int_text[:max_len]) # On remplit la rangée correspondante (et on coupe à max_len tokens)

# Même chose pour Y
Y = torch.zeros(len(texts), max_len, dtype=torch.long)
for i, int_label in enumerate(int_labels):
    if len(int_label) < max_len:
        int_label = int_label + [len(label2int)] * (max_len - len(int_label))
    Y[i] = torch.LongTensor(int_label[:max_len])

print(X.size())
print(Y.size())

In [108]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=token2int['<eos>'])
        self.embed.weight = nn.Parameter(embeddings, requires_grad=False)
        self.rnn = nn.GRU(embed_size, hidden_size, bias=False, num_layers=1, bidirectional=False, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.decision = nn.Linear(hidden_size * 1 * 1, len(label2int))
        
    def forward(self, x):
        embed = self.embed(x)
        output, hidden = self.rnn(embed) # On veut appeler notre réseau de neurones sur la sortie de la couche précédente
        # c-a-d les embeddings qu'on vient de créer à la ligne du dessus
        return self.decision(self.dropout(output))

rnn_model = RNN()
rnn_model

NameError: name 'vocab_size' is not defined

In [ ]:
def __getitem__(index):
        """Return a data sample (=image) for a given index, along with the name of the corresponding pokemon."""
        
        # TO DO:
        # - get the image path corresponding to 'index' (use the list 'self.image_path_list')
        # - get the pokemon name
        # - load the image into a numpy array x
        # - transform x into a pytorch tensor of type 'float'
        # - return the tensor x and the pokemon name
#         image_path = image_path_list[index]
#         label = image_path.split('/')[-2]
#         label = self.class_to_idx[label]
#         x = io.imread(image_path)
#         x = torch.tensor(x, dtype=float)
        
        return x, label

In [ ]:
tokens = []
labels = []
token_label = []
for word in lines:
    word = word.replace('\n', '')
    token_label.append(word.split('\t'))
# #     print(token_label[-1])
#     labels.append(token_label[1])
#     tokens.append(token_label[0])

In [ ]:
# nlp = spacy.load("en_core_web_sm")

In [ ]:
# #Code to get tokenized/segmented text:
# nlp = spacy.load("en_core_web_sm")
# doc = nlp(text10)
# for sent in doc.sents:
#     for token in sent:
#         print(token.text+"\tO")
#     print()

In [ ]:
class XrayDataset(Dataset):

    def __init__(self, image_dir):
        """Initialize the attributes of the object of the class."""
        self.image_dir = image_dir
        self.image_path_list = sorted(self._find_files(image_dir))
        self.classes = self._get_class()
        self.class_to_idx  = self._get_class_index()

    def __len__(self):
        """Return the size of the dataset."""
        return len(self.image_path_list)

    def __getitem__(self, index):
        """Return a data sample (=image) for a given index, along with the name of the corresponding pokemon."""
        
        # TO DO:
        # - get the image path corresponding to 'index' (use the list 'self.image_path_list')
        # - get the pokemon name
        # - load the image into a numpy array x
        # - transform x into a pytorch tensor of type 'float'
        # - return the tensor x and the pokemon name
        image_path = self.image_path_list[index]
        label = image_path.split('/')[-2]
        label = self.class_to_idx[label]
        x = io.imread(image_path)
        x = torch.tensor(x, dtype=float)
        
        return x, label

    def _find_files(self, directory, pattern='*.jpeg'):
        """Recursively find all files matching the pattern."""
        image_path_list = []
        for root, dirnames, filenames in os.walk(directory):
            for filename in fnmatch.filter(filenames, pattern):
                image_path_list.append(os.path.join(root, filename))
        return image_path_list
    
    def _get_class(self):
        classes = []
        for path in self.image_path_list:
            class_image = path.split('/')[2]
            if class_image not in classes:
                classes.append(class_image)
        return classes
    
    def _get_class_index(self):
        idx_to_class = {i:j for i, j in enumerate(self.classes)}
        class_to_idx = {value:key for key,value in idx_to_class.items()}
        return class_to_idx

In [ ]:
# We can initialize the dataset by providing the directory of the image corpus data
image_dir = 'Dataset/train'
dataset = XrayDataset(image_dir=image_dir)

# You can use the '_find_files' method to get the list of images paths
image_path_list = dataset._find_files(image_dir)

# Equivalently, since this list is stored as an inner attribute, you can access it directly:
image_path_list = dataset.image_path_list

# Display all file paths
for l in image_path_list:
    print(l)
print('Number of images in the list', len(image_path_list))

In [ ]:
train_dir = 'Dataset/train'
val_dir = 'Dataset/val'
test_dir = 'Dataset/test'
train_data = XrayDataset(image_dir=train_dir)
val_data = XrayDataset(image_dir=val_dir)
test_data = XrayDataset(image_dir=test_dir)

In [ ]:
# Set the batch size (=number of samples/images in each batch) and create the dataloader
batch_size = 5
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
train_dataloader